In this example I implement in Keras the simple Neural Network solved in the other
scripts by using my Monte Carlo strategy. The idea is: Keras is able to build NN very easily.
My Monte Carlo method needs to be tested on more realistic networks.
Therefore: I find a way to implement my MC startegies directly
on Keras.

In [15]:
import numpy as np
import keras
from keras.models import Sequential 
from keras.layers import Dense, Activation

In [16]:
# Define the training set as in the example
# Points to classify
x = np.array([[0.1, 0.1], [0.3, 0.4], [0.1, 0.5], [0.6, 0.9], [0.4, 0.2],
                [0.6, 0.3], [0.5, 0.6], [0.9, 0.2], [0.4, 0.4], [0.7, 0.6]])
# Their labels
y = np.array([[1,0], [1,0], [1,0], [1,0], [1,0], [0,1], [0,1], [0,1], [0,1],
                    [0,1]])

In [17]:
## Define my Keras neural network
model = Sequential()
model.add(Dense(2, input_dim = 2, activation='sigmoid'))
model.add(Dense(3, activation = 'sigmoid'))
model.add(Dense(2, activation = 'sigmoid'))

We have now important global variables: x, y (the training data), and model, i.e.
the Neural Network model offered by keras.

I want now a function such that, given an array of the right lenght,
compute the mean squared error of the keras model.
The right lenght is precisely the number of the parameters of the model, 
given by the function below:

In [61]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 8         
Total params: 23
Trainable params: 23
Non-trainable params: 0
_________________________________________________________________


The following section is a collection of auxhiliary functions.

In [62]:
# Function to get the total element of an array which is linear or matrix
def tot_elm(var):
    try:
        res = var.shape[0] * var.shape[1]
    except:
        res = var.shape[0]
    return res

In [63]:
# Given a generic numpy array of the right size, fit it into the
# right format to be used as a parameter for the keras Neural Network
def params_to_keras(p):
    offset = 0
    q = model.get_weights()
    for i in range(len(q)):
        take_n = tot_elm(q[i]) 
        q[i] = np.asanyarray(p[offset:offset+take_n]).reshape(q[i].shape)
        offset += take_n
    return q

In [64]:
# Given a generic 23-dim numpy array, compute the model cost
# function according to the keras model
def keras_cost(p):
    p = params_to_keras(p)
    model.set_weights(p)
    y_hat = model.predict(x)
    mse = keras.losses.MeanSquaredError()
    return mse(y, y_hat).numpy() * 100 

REMARK: keras_cost is now PRECISELY the function that we want to minimize
by using the Monte Carlo approach!!!! Very well.

In [65]:
# Just playing a bit with random coefficients
myparams = np.random.uniform(0, 1, 23)
for_keras = params_to_keras(myparams)

In [66]:
print(myparams)

[0.36567339 0.49332022 0.63958298 0.54891628 0.95097856 0.08137408
 0.06314725 0.28708884 0.25873719 0.39525301 0.85979856 0.93944953
 0.04905192 0.83884441 0.1718297  0.79993486 0.62376031 0.40022533
 0.05828978 0.97013973 0.07194908 0.01944271 0.31980972]


In [67]:
print(for_keras)

[array([[0.36567339, 0.49332022],
       [0.63958298, 0.54891628]]), array([0.95097856, 0.08137408]), array([[0.06314725, 0.28708884, 0.25873719],
       [0.39525301, 0.85979856, 0.93944953]]), array([0.04905192, 0.83884441, 0.1718297 ]), array([[0.79993486, 0.62376031],
       [0.40022533, 0.05828978],
       [0.97013973, 0.07194908]]), array([0.01944271, 0.31980972])]


In [68]:
keras_cost(myparams)

31.943267583847046

In other words: INSTEAD of using my "old" nn_potential, with all the Neural Network
functions written by hands, I can use now the Keras interface directly.
That's very, very nice since provides huge flexibility.
Despite that, please note:

We do not have no more the explicit gradient descent (so...good that I worked with RW
Monte Carlo only!) and that here there is no plot function as in my general library.
No problems at all: all functions that will be added. 

FINAL GOAL: to have something like:

In [70]:
#model.compile(optimizer = 'my_monte_carlo_approach') # :-)